In [108]:
from torchvision import datasets as ds
from torch.utils.data import DataLoader
from torchvision import transforms as ts
import torchvision as tv
import torch
import torch.nn as nn
import math
import numpy as np
from torch.autograd import Variable
from torch import optim
from matplotlib import pyplot as plt
import torch.backends.cudnn as cudnn
import timm
from PIL import Image
from tqdm import tqdm
import 

In [ ]:
def set_seed(self, seed):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)

## Dataset

In [2]:
transform = ts.Compose(
    [
        ts.ToTensor(),
        ts.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [125]:
target_transform = ts.Compose([ts.ToTensor()])

In [3]:
train_set = ds.CIFAR10(root='../data/', train=True, transform=transform, download=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../data/cifar-10-python.tar.gz to ../data/


In [111]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=200, shuffle=True, num_workers=0)

In [97]:
val_set = tv.datasets.CIFAR10(root='../data/',train=False, download=True, transform=transform)

Files already downloaded and verified


In [112]:
val_loader = torch.utils.data.DataLoader(val_set, batch_size=200, shuffle=True, num_workers=0)

In [42]:
def make_test_list(data_dir):
    
    test_img_list = list()
    
    files = os.listdir(data_dir)
    
    for i in range(10):
        img_file = data_dir + "/" + files[i]
        img = os.listdir(img_file)
        for j in range(200):
            img_path = data_dir + "/" +  files[i] + "/" + img[j]
            
            test_img_list.append(img_path)
    return test_img_list

In [43]:
img_list = make_test_list("../Statistical_Deep_Image_2021/Statistical_Deep_Image")

In [56]:
len(img_list)

2000

In [166]:
def make_test_label(label_list):
    
    test_label_list = list()
    
    for i in range(10):
        for j in range(200):
            test_label_list.append(i)
        
    return test_label_list

In [167]:
label = os.listdir("../Statistical_Deep_Image_2021/Statistical_Deep_Image")
img_label_list = make_test_label(label)

In [57]:
len(img_label_list)

2000

In [178]:
class testset(torch.utils.data.Dataset):
    def __init__(self, img_list, img_label_list, transform, target_transform):
        self.file_list = img_list
        self.labels = img_label_list
        self.transform = transform
        self.target_transform = target_transform
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        return img_transformed, self.labels[index]

In [179]:
test_set = testset(img_list = img_list,
                  img_label_list = img_label_list,
                  transform = transform,
                  target_transform = target_transform)

In [180]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 200, shuffle=True, num_workers=0)

In [14]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Model training

In [91]:
model = timm.create_model('vgg16', pretrained=False, num_classes=10)

In [115]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [117]:
for epoch in tqdm(range(2)):
    
    running_loss = 0.0
    
    for i, data in tqdm(enumerate(train_loader, 0)) :
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f'epoch: {epoch + 1} loss: {running_loss / 500:.3f}')
        

  0%|          | 0/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:04,  4.87s/it]
2it [00:09,  4.99s/it]
3it [00:15,  5.04s/it]
4it [00:19,  4.96s/it]
5it [00:24,  5.00s/it]
6it [00:29,  4.97s/it]
7it [00:34,  4.96s/it]
8it [00:39,  4.85s/it]
9it [00:44,  4.88s/it]
10it [00:49,  4.85s/it]
11it [00:54,  4.92s/it]
12it [00:59,  4.89s/it]
13it [01:03,  4.85s/it]
14it [01:08,  4.94s/it]
15it [01:14,  4.97s/it]
16it [01:19,  5.02s/it]
17it [01:24,  5.00s/it]
18it [01:28,  4.91s/it]
19it [01:33,  4.92s/it]
20it [01:38,  4.94s/it]
21it [01:43,  4.90s/it]
22it [01:48,  4.90s/it]
23it [01:53,  4.94s/it]
24it [01:58,  4.97s/it]
25it [02:03,  4.98s/it]
26it [02:08,  4.99s/it]
27it [02:13,  4.91s/it]
28it [02:18,  4.92s/it]
29it [02:23,  4.92s/it]
30it [02:28,  4.94s/it]
31it [02:33,  4.95s/it]
32it [02:38,  4.97s/it]
33it [02:43,  4.95s/it]
34it [02:47,  4.96s/it]
35it [02:52,  4.91s/it]
36it [02:57,  4.92s/it]
37it [03:02,  5.00s/it]
38it [03:07,  4.90s/it]
39it [03:12,  4.84s/it]
40it [03:17,  4.

epoch: 1 loss: 0.681



1it [00:05,  5.10s/it]
2it [00:10,  5.13s/it]
3it [00:15,  5.07s/it]
4it [00:20,  5.05s/it]
5it [00:25,  5.02s/it]
6it [00:30,  4.97s/it]
7it [00:34,  4.89s/it]
8it [00:39,  4.87s/it]
9it [00:44,  4.87s/it]
10it [00:49,  4.90s/it]
11it [00:54,  4.98s/it]
12it [00:59,  4.99s/it]
13it [01:04,  4.93s/it]
14it [01:09,  4.92s/it]
15it [01:14,  4.94s/it]
16it [01:19,  4.99s/it]
17it [01:24,  5.02s/it]
18it [01:29,  5.00s/it]
19it [01:34,  5.02s/it]
20it [01:39,  5.04s/it]
21it [01:44,  5.00s/it]
22it [01:49,  4.95s/it]
23it [01:54,  4.99s/it]
24it [01:59,  4.98s/it]
25it [02:04,  5.03s/it]
26it [02:09,  4.98s/it]
27it [02:14,  5.00s/it]
28it [02:19,  5.00s/it]
29it [02:24,  5.03s/it]
30it [02:29,  5.03s/it]
31it [02:34,  5.00s/it]
32it [02:39,  4.94s/it]
33it [02:44,  4.96s/it]
34it [02:49,  4.98s/it]
35it [02:54,  4.97s/it]
36it [02:59,  4.99s/it]
37it [03:04,  4.93s/it]
38it [03:08,  4.86s/it]
39it [03:14,  5.18s/it]
40it [03:20,  5.20s/it]
41it [03:24,  5.10s/it]
42it [03:29,  5.07s/it]


epoch: 2 loss: 0.674


## test

In [182]:
correct = 0
total = 0

with torch.no_grad():
    for data in tqdm(test_loader):
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy: %d %%' % (100 * correct / total))

100%|██████████| 10/10 [00:07<00:00,  1.35it/s]

Accuracy: 14 %
